# [Notebook 2]: Inference Phase (Multi-modal RAG)
For the indexing phase, see notebook "Build-vector-store-and-index"

In [0]:
%pip install openai --upgrade --quiet
%pip install mlflow[genai]>=2.9.0 --quiet
%pip install FPDF --quiet
%pip install --upgrade pydantic --quiet
%pip install --upgrade gradio --quiet
%pip install databricks.vectorsearch --quiet

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.r

In [0]:
from openai import OpenAI

DATABRICKS_ACCESS_TOKEN = f"YOUR_DATABRICKS_TOKEN"

client = OpenAI(
  api_key=DATABRICKS_ACCESS_TOKEN,
  base_url="https://dbc-f499a870-66c0.cloud.databricks.com/serving-endpoints"
)

First load the same CLIP model that we used in the indexing phase

In [0]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np

MODEL = 'openai/clip-vit-large-patch14'

clip_model = CLIPModel.from_pretrained(MODEL)
clip_processor = CLIPProcessor.from_pretrained(MODEL)

2024-05-06 16:36:15.526283: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Then access the vector search endpoint and index

In [0]:
from databricks.vector_search.client import VectorSearchClient

VECTOR_SEARCH_ENDPOINT_NAME = 'YOUR_ENDPOINT'
VS_INDEX_FULLNAME = 'YOUR_INDEX'
vsc = VectorSearchClient()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Build functions to get the CLIP embedding, retrieve relevant logs, build the final prompt and send to GPT4-Vision. Also function for creating the final PDF.

In [0]:
import base64
import io
from fpdf import FPDF
import tempfile
import datetime

def retrieve_and_prepare_prompt(user_text, logs):
    combined_full_logs = "\n\n".join([f"LOG {i+1}:\n{logs}" for i, logs in enumerate(logs)])
    
    # Prepare the prompt for GPT-4-Vision
    prompt = f'''
    There is a field service engineer and have been sent out to inspect a manufacturing-based machine out int he field and provide a maintenance log. He jotted down a few notes but wants you to write out the full log.

    He has given you an image of the machine and his jotted-down notes.

    Luckily, you have access to prior maintenance logs and associated images. Based on the image and his notes, you have retrieved the following full logs that are most similar to the combination of image and notes he has given you.

    Based on what you see in the logs, plus the attached image, plus the jotted down notes below, write a full maintenance log in the same format as the logs.

    Each log has to have the 3 sections: 1) Inspection Summary, 2) Technical Evaluation, 3) Recommendations.

    =====
    
    Image: (attached)
    User notes: {user_text}\n
    
    Combined full logs of similar prior entries: \n
    {combined_full_logs}
    
    ====
    Generate a full log based on the above information that resembles the format of the combined full logs given and leverages the text in those logs plus the attached image to help you generate this new log. Use the retrieved logs mostly for informing how you format your generated log and if there are relevant pieces of information in those logs that can help you, feel free to leverage them but make sure you place a lot of emphasis on the attached image.
    '''
    return prompt

def get_clip_embedding(text, image_input):
    # Check if the input is a NumPy array
    if isinstance(image_input, np.ndarray):
        # Convert NumPy array to PIL Image
        image = Image.fromarray(np.uint8(image_input)).convert('RGB')
    else:
        # Load image from the path
        image = Image.open(image_input).convert('RGB')
    
    inputs = clip_processor(text=[text], images=[image], return_tensors="pt", padding=True, truncation=True)
    outputs = clip_model(**inputs)
    image_features = outputs.image_embeds  # This should be the embedding for the image
    text_features = outputs.text_embeds    # This should be the embedding for the text
    combined_embedding = (image_features + text_features) / 2
    return combined_embedding.squeeze().detach().numpy()

def get_relevant_docs(embedding):
    query_vector = embedding.tolist() if isinstance(embedding, np.ndarray) else embedding
    results = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, VS_INDEX_FULLNAME).similarity_search(
        query_vector=query_vector,
        columns=["log"],
        num_results=3)
    docs = results.get('result', {}).get('data_array', [])
    return docs

def encode_the_image(image_input):
    # Check if the input is a NumPy array
    if isinstance(image_input, np.ndarray):
        # Convert NumPy array to PIL Image
        image = Image.fromarray(np.uint8(image_input)).convert('RGB')
    else:
        # Load image from the path
        image = Image.open(image_input).convert('RGB')
    
    buffer = io.BytesIO()
    image.save(buffer, format="PNG")  # Saving the image in PNG format
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    return img_base64

def get_gpt4_generate_log(client, prompt, base64_encoded_image):
    chat_completion = client.chat.completions.create(
    messages=[
        {
        "role": "system",
        "content": "You are an AI assistant"
        },
        {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_encoded_image}"
                }
            }
        ]
        }
    ],
    model="ak-openai-gpt4v",
    max_tokens=4096
    )
    # print(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

def generate_log_for_user(client, image, notes):
    try:
        print("Getting embedding ...")
        embedding = get_clip_embedding(notes, image)  # Ensure this function is properly defined and imported
        print("Getting similar logs ...")
        similar_logs = get_relevant_docs(embedding)  # Ensure this function is properly defined and imported
        print("Encoding image ...")
        base64_encoded_image = encode_the_image(image)  # Ensure this function is properly defined and imported
        print("Building full prompt ...")
        full_prompt = retrieve_and_prepare_prompt(notes, similar_logs)  # Ensure this function is properly defined and imported
        print("Generating log ...")
        generated_log = get_gpt4_generate_log(client, full_prompt, base64_encoded_image)  # Ensure this function is properly defined and imported
        return generated_log
    except Exception as e:
        print(f"Error in generate_log_for_user: {e}")
        return str(e)  # Return error as string for debugging purposes

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, 'Maintenance log 5/5/24', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(2)  # Small line break after the title

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 5, body, align='L')
        self.ln()

    def add_image(self, image_input):
        if isinstance(image_input, np.ndarray):
            image = Image.fromarray(image_input.astype('uint8')).convert('RGB')
        else:
            image = Image.open(image_input).convert('RGB')
        
        with tempfile.NamedTemporaryFile(suffix=".jpg", delete=False) as tmp_image:
            image.save(tmp_image, format="JPEG")
            tmp_image.flush()  # Ensure all data is written to file
            x_center = (210 - 100) / 2  # Center the image
            self.image(tmp_image.name, x=x_center, w=100)
            self.ln(10)  # Space after the image

def create_pdf(image_input, generated_log, notes):
    pdf = PDF()
    pdf.add_page()
    pdf.add_image(image_input)

    # Custom parsing and formatting of the log text
    sections = ["Inspection Summary", "Technical Evaluation", "Recommendations"]
    start = 0
    for section in sections:
        start_idx = generated_log.find(section, start)
        if start_idx != -1:
            end_idx = generated_log.find("\n", start_idx)
            if end_idx == -1:
                end_idx = len(generated_log)
            pdf.chapter_title(generated_log[start_idx:end_idx])
            body_start = end_idx + 1
            body_end = min([generated_log.find(s, body_start) for s in sections if generated_log.find(s, body_start) != -1] + [len(generated_log)])
            pdf.chapter_body(generated_log[body_start:body_end].strip())
            start = body_end

    # Dynamic filename generation with timestamp
    current_time = datetime.datetime.now().strftime("%H-%M-%S")
    pdf_output = f"./pdf_{current_time}.pdf"
    pdf.output(pdf_output)
    return pdf_output

Creates Gradio app for use within Databricks

In [0]:
import gradio as gr
import time

from IPython.display import display, HTML

# Custom CSS to adjust the iframe width and height within the Jupyter notebook
custom_css = """
<style>
    iframe { width: 100% !important; height: 900px !important; }
</style>
"""

# Display the custom CSS
display(HTML(custom_css))

with gr.Blocks(theme=gr.themes.Soft()) as app:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="numpy", label="Upload Image")
            text_input = gr.Textbox(label="Raw maintenance notes", placeholder="Enter maintenance notes here...")
        with gr.Column():
            log_output = gr.Textbox(label="Generated log shows up here", interactive=False, value="", placeholder="...")
    
    generate_log_button = gr.Button("Generate Log")
    download_pdf_button = gr.Button("Download PDF")

    def on_download_pdf(image, log, notes):
        time.sleep(2)
        pdf_content = create_pdf(image, log, notes)
        return pdf_content
    
    generate_log_button.click(
        generate_log_for_user,
        inputs=[client, image_input, text_input],
        outputs=log_output
    )
    
    download_pdf_button.click(
        on_download_pdf,
        inputs=[image_input, log_output, text_input],
        outputs=None
    )

    with gr.Row():
        generate_log_button
        download_pdf_button

app.launch(
    share=True,
    inline=True,            # Ensure the interface launches within the notebook cell
    width='100%',           # Set the width to 100% of the cell
    height=800             # Adjust the height to avoid scrolling (set as per your content)
)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://b9c8ee6717ceb38b7a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
